# 0. Prerequisites

## 0-1. Requirements

* Ubuntu 18.04, Cuda 11
* opencv-python
* numpy
* pandas
* timm
* torch==1.8.0 torchvision 0.9.0 with cuda 11.1
* natsort
* scikit-learn==1.0.0
* pillow
* torch_optimizer
* tqdm
* ptflops
* easydict
* matplotlib

## 0-2 Directory 구조


In [ ]:
import os

def list_files(startpath):
  for root, dirs, files in os.walk(startpath):
    level = root.replace(startpath, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)
    for f in files:
      print('{}{}'.format(subindent, f))
list_files('../')

## 0-3. Import Library

In [ ]:
!pip install ptflops
!pip install timm
!pip install torch_optimizer

     |████████████████████████████████| 431 kB 12.5 MB/s 
     |████████████████████████████████| 61 kB 452 kB/s 


In [ ]:
import os
import cv2
import json
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob # 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
from pathlib import Path
from natsort import natsorted # 텍스트로 된 숫자(파일명) 정렬
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from PIL import Image

import timm # PyTorch Image Models
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings("ignore")

### 0-4. Config


모델 및 학습의 Hyper-parameter 정의 (사용자에 의해 세팅되는 값)

In [ ]:
# EasyDict는 자바스크립트와 같이 dict의 value에 속성으로 접근할 수 있다. 
args = easydict.EasyDict(
    {'exp_num':'0',
     'experiment':'Base',
     'tag':'Default',

     # Path settings
     'data_path': '../data',
     'fold':4, 
     'Kfold':5,
     "model_path":'results/',

     # Model parameter settings
     "encoder_name":'regnety_040',
     "drop_path_rate":0.2,

     # Training parameter settings
     ## Base Parameter
     "img_size":288, 
     "batch_size":16, 
     "epochs":60,
     "optimizer":"Lamb",
     "initial_lr":5e-6,
     "weight_decay":1e-3,

     ## Augmentation
     "aug_ver":2, 
     "flipaug_ratio":0.3, 
     "margin":50, 
     "random_margin":True, 

     ## Scheduler
     "scheduler":"cycle",
     "warm_epoch":5, 
     ### Cosine Annealing
     "min_lr":5e-6,
     "tmax":145, 
     ### OneycleLR
     'max_lr':1e-3, 

     ## etc
     'patience':50, 
     'clipping':None,

     # Hardware settings
     'amp':True, 
     'multi_gpu':False, 
     'logging':False, 
     'num_workers':4, 
     'seed':42
    }
)

# 1. Make dataset


In [ ]:
# keypoint를 기준으로 이미지를 crop하기 위한 함수 정의 
# train과 test 시 해당 함수가 적용된 crop 이미지가 Inputs으로 들어가게 된다. 

def crop_image(images, point, margin=100):
  image = np.array(Image.open(images).convert('RGB')) # 이미지 객체를 RGB로 변환하기
  point = point['data']
  max_point = np.max(np.array(point), axis=0).astype(int)+margin 
  min_point = np.min(np.array(point), axis=1).astype(int)-margin
  max_point = max_point[:-1] # remove Z order
  min_point = min_point[:-1] # remove Z order

  max_x, max_y = max_point
  min_x, min_y = min_point
  max_y += margin # 손목까지

  # 데이터 포인트의 크기가 원 이미지를 넘어서는 경우를 방지
  max_x = max_x if max_x < 1920 else 1920
  max_y = max_y if max_y < 1080 else 1080
  min_x = min_x if min_x > 0 else 0
  min_y = min_y if min_y > 0 else 0

  crop_image = image[min_y:max_y, min_x:max_x]

  return crop_image

In [ ]:
# dataloader에서 사용할 dataframe 만들기
train_path = '../data/train'
train_folders = natsorted(glob(train_path + "/*"))

answers=[]
for train_folder in train_folders:
  json_path = glob(train_folder+"/*.json")[0]
  js = json.load(open(json_path))
  cat = js.get("action")[0]
  cat_name = js.get('action')[1]

  images_list= glob(train_folder +"/*.png")
  for image_name in images_list:
    answers.append([image_name, cat, cat_name])

answers = pd.DataFrame(answers, columns = ['train_path','answer','answer_name'])
answers.to_csv("../data/df_train.csv", index=False)

# 클래스가 1개뿐인 폴더들 Augmentation해서 이미지 생성 후, dataframe 재정의.
# 새롭게 정의한 dataframe을 학습에 이용시 약간의 성능향상을 확인할 수 있음. 
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

data_path = "../data"
df_train = pd.read_csv(opj(data_path, 'df_train.csv'))
df_info = pd.read_csv(opj(data_path, "hand_geture_pose.csv"))
df_train =pd_train.merge(df_info[['pose_id', "gesture_type","hand_type"]], 
                         how="left", left_on="answer", right_on ="pose_id")
save_folder = "train"
for i in range(649, 649+5):
  if not os.path.exists(opj(data_path, save_folder, str(i))):
    os.makedirs(opj(data_path, save_folder, str(i)))

# flip aug 가능한 lable: 131, 47(one sample)
oslabel_fliplabel = [(131,156), (47, 22)] # one sample label, flip label
folders = ['649', '650'] # Train 648번 folder에 이은 number 생성

# tqdm: 진행률 프로세스 바
# zip은 순회가능 한 객체를 결합해 줌 ex. zip([1,2,3],['A','B','C']) => [(1,'A'),(2,'B'),(3,'C')]
for label, folder in tqdm(zip(oslabel_fliplabel, folders)):
  idx=0
  os_label, f_label = label[0],label[1]
  one_sample = df_train[df_train['answer'] == os_label].reset_index(drop=True)
  temp = df_train[df_train['answer']== f_label].reset_index(drop=True)
  train_folders = natsorted(temp['train_path'].apply(lambda x:x[:-6]).unique())

  for train_folder in (train_folders):
    json_path = glob(train_folder + "/*.json")[0]
    js = json.load(open(json_path))
    keypoints = js['annotations']
    images_list = natsorted(glob(train_folder + "/*.png"))
    for _, (point, image_name) in enumerate(zip(keypoints, images_list)):
      croped_image = crop_image(image_name, point, margin= 50)
      flip_img = cv2.flip(croped_image, 1)
      save_path = opj(data_path, save_folder, folder, f'{idx}.png')
      idx += 1
      cv2.imwrite(save_path, flip_img)
      df_train.loc[len(df_train)] = [save_path] + one_sample.iloc[0][1:].values.tolist()

def rotation(img, angle):
  angle = int(random.uniform(-angle, angle))
  h, w = img.shape[:2]
  M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
  img = cv2.warpAffine(img, M, (w,h))
  return img

oslabel=[92, 188, 145]
folder = ['651','652','653']
for label, folder in tqdm(zip(oslabel, folder)):
  idx=0
  one_sample = df_train[df_train['answer']==label].reset_index(drop=True)
  train_folders = natsorted(temp['train_path'].apply(lambda x:x[:-6]).unique())
  for train_folder in (train_folders):
    json_path = glob(train_folder+"/*.json")[0]
    js =json.load(opne(json_path))
    keypoints = js['annotations']
    images_list = natsorted(glob(train_folder + "/*.png"))
    for _, (point, image_name) in enumerate(zip(keypoints, images_list)):
      croped_image = crop_image(image_names, point, margin =50)
      aug_img = rotation(croped_image, 30)
      save_path = opj(data_path, save_folder, folder, f'{idx}.png')
      idx +=1
      cv2.imwrite(save_path, aug_img)
      df_train.loc[len(df_train)] = [save_path] + one_sample.iloc[0][1:].values.tolist()

df_train.to_csv("../data/df_train_add.csv", index=False)

## Dataset & Loader (Flip Augmentation & Crop using the keypoints & Remove noise keypoints)

original image를 불러 다양한 Margin 값에 대해 crop 하는 방식으로 pytorch의 dataset을 구성함.

In [ ]:
# Train dataset에 475, 543 폴더는 의도하지 않은 나머지 손에 대해서도 Keypoint가 잡하게 됨. 
# Json의 Keypoint를 사용하기 위해 475, 543 폴더인 경우 해당 부분 Keypoint 제거

def remove_keypoints(folder_num, points):
  lst = []
  for x,y,z in points:
    cond1 = x < 250 and y > 800
    cond2 = x > 1400 and y < 400
    if not (cond1 or cond2):
      lst.append([x,y,z])
  return lst

class Train_Dataset(Dataset):
  def __init__(self, df, transform = None, df_flip_info=None, flipaug_ratio=0, label_encoder=None, margin=50, random_margin =True):
    self.id = df['train_path'].values
    self.target= df['answer'].values
    self.transform = transform
    self.margin = margin
    self.random_margin = random_margin

    # Flip Augmentation (Change target class)
    if df_flip_info is not None:
      self.use_flip = True
      print("Use Flip Augmentation")
      left = label_encoder.transform(df_flip_info['left'])
      right = label_encoder.transform(df_flip_info['right'])
      left_to_right = dict(zip(left, right))
      right_to_left = dict(zip(right, left))

      self.flip_info = left_to_right.copy()
      self.flip_info.update(right_to_left)
      self.flip_possible_class  = list(set(np.concatenate([left, right])))
    self.flipaug_ratio = flipaug_ratio

    print(f'Dataset size:{len(self.id)}')

  def __getitem__(self, idx):
    image = np.array(Image.open(self.id[idx]).convert("RGB"))
    target = self.targe[idx]

    # Load Json File
    try:
      image_num = int(Path(self.id[idx]).stem)
      dir = os.path.dirname(self.id[idx])
      folder_num = os.path.basename(dir)
      json_path = opj(dir, folder_num + ".json")
      js = json.load(open(json_path))
      keypoints = js['annotations'][image_num]['data'] # 해당 이미지에 해당하는 Keypoints
    except: # Augmentation으로 직접 새로 만든 foldersms Json이 없으므로 바로 return 
      image = self.transform(Image.fromarray(image))
      return image, np.array(target)

    if folder_num in ['475', '543']:
      keypoints = remove_keypoints(folder_num, keypoints)

    # Image Crop using keypoints
    max_point = np.max(np.array(keypoints), axis=0).astype(int) + self.margin
    min_point = np.min(np.array(keypoints), axis=0).astype(int) - self.margin
    max_point = max_point[:-1] #remove Z order
    min_point = min_point[:-1] #remove Z order

    max_x, max_y = max_point
    min_x, min_y = min_point
    max_y += 100 # 손목부분까지 여유

    # 매 epoch마다 margin이 조금씩 다르게 들어가므로 한 폴더 내 비슷한 이미지들의 overfitting을 방지 (only train phase)
    if self.random_margin:
      if random.random() < 0.5:
        max_x += self.margin
      if rnadom.random() < 0.5:
        max_y += self.margin
      if random.random() < 0.5:
        min_x -= self.margin
      if random.random() < 0.5:
        min_y -= self.margin
    else:
      max_x += self.margin
      max_y += self.margin
      min_x -= self.margin
      min_y -= self.margin

    #데이터 포인트의 크기가 원 이미지를 넘어서는 경우 방지
    max_x = max_x if max_x < 1920 else 1920
    max_y = max_y if max_y < 1080 else 1080
    min_x = min_x if min_x > 0 else 0
    min_y = min_y if min_y > 0 else 0

    image = image[min_y:max_y, min_x:max_x]

    # FlipAug
    if(random.random() < self.flipaug_ratio) and (target in self.flip_possible_class):
      image = np.flip(image, axis=1) # (H, W, C)에서 width 축 flip
      target = self.flip_info[target]
    image = self.transform(Image.fromarray(image))
    return image, np.array(target)

  def __len__(self):
    return len(self.id)
  
  
def get_loader(df, batch_size, shuffle, num_workers, transform , df_flip_info = None,
               flipaug_ratio=0, label_encoder = None, margin = 50, random_margin =True):
  dataset = Train_Dataset(df, transform, df_flip_info=df_flip_info, flipaug_ratio=flipaug_ratio, 
                            label_encoder=label_encoder, margin=margin, random_margin=random_margin)
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                drop_last=False)
  return data_loader


def get_train_augmentation(img_size, ver):
  if ver==1:
    # For Test
    transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),
            ])
  if ver==2:
    # For Train
    transform = transforms.Compose([
            transforms.RandomAffine(20),
            transforms.RandomPerspective(),
            transforms.ToTensor(),
          transforms.Resize((img_size, img_size)),
          transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
        ])
return transform

### Flip Augmentation

* 중복된 데이터가 많아 오버피팅이 빠르게 발생하여 적절한 Augmentation 수행 필요
* 평소 이미지 Task에서 흔하게 사용되는 Augmentationdls Horizontal Flip은 사용할 수 없었음. (동일한 포즈라도 왼손, 오른손 Class가 다르기 때문)
* 동일한 포즈일 때, 이미지는 Horizontal Flip을 시키고 Class 도 바꾸어주면 충분히 학습에 사용할 수 있을 것이라고 판단.
* 예로, My View, 왼손, 숫자 1일 때 HFlip을 수행하여 My View, 오른손, 숫자 1 클래스를 생성
* 이를 위해 미리 아래와 같으 mapping dataFrame 을 만들어 주었고 pytorch dataset안에 이식하였음
* Flip augmentation을 수행하는 비율을 0.1~0.5로 다양하게 해보았을 때 0.3이 가장 적절하였음.


### Network


pytorch image model(timm) 라이브러리를 활용하여 generalization performance에 강점을 가지는 RegNet 을 Base 모델로 사용

In [ ]:
class Pose_Network(nn.Module):
  def __init__(self, args):
    super().__init__()
    self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                     drop_path_rate = args.drop_path_rate)
    num_head = self.encoder.head.fc.in_features
    self.encoder.head.fc = nn.Linear(num_head, 157)

  def forward(self,x):
    return self.encoder(x)

NameError: ignored

### Utils for training and Logging

logging과 avgmeter를 이용해 실험 기록을 log파일로 남도록 저장한다. 

* leargning rate scheduler
* 학습과정에서 lr을 조정함.

In [ ]:
# warmup learning rate scheduler
from torch.optim.lr_scheduler import _LRScheduler
class WarnUpLR(_LRScheduler):
  """warmup_training learning rate scheduler
  Args:
    optimizer: optimizer(e.g.SGD)
    total_iters: totla_iters of warmup phase
  """
  def __init__(self, optimizer,total_iters, last_epoch=-1):
    self.total_iters = total_iters
    super().__init__(optimizer, last_epoch)

  def get_lr(self):
    """ we will use the first m batches, and set the learning rate to base_lr * m/ total_iters"""
    return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

#Logging
def get_root_logger(logger_name="basicsr", log_level= logging.INFO, log_file =None):
  logger = logging.getLogger(logger_name)
    # if the logger has been initialized, just return it
  if logger.hasHandlers():
      return logger

  format_str = '%(asctime)s %(levelname)s: %(message)s'
  logging.basicConfig(format=format_str, level=log_level)

  if log_file is not None:
      file_handler = logging.FileHandler(log_file, 'w')
      file_handler.setFormatter(logging.Formatter(format_str))
      file_handler.setLevel(log_level)
      logger.addHandler(file_handler)

  return logger

class AvgMeter(object):
  def __init__(self):
    self.reset()

  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0
    self.losses = []

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count
    self.losses.append(val)

### Trainer
모델의 학습(training function)과 검증(ValidatioN)을 위한 Class이다. 

In [ ]:
class Trainer():
  def __init__(self, args, save_path):
    """
    args: arguments
    save_path: Model 가중치 저장 경로 
    """
    super(Trainer, self).__init__()
    self.device = torch.device('cuda'if torch.cuda.is_available()else"cpu")

    #logging
    log_file = os.path.join(save_path, "log.log")
    self.logger = get_root_logger(logger_name="IR", log_level=loggin.INFO, log_file=log_file)
    self.logger.info(args)
    self.logger.info(args.tag)

    # Train, Valid Set load
    ################
    df_train = pd.read_csv(opj(args.data_path, 'df_tain_add.csv'))
    df_info = pd.read_csv(opj(args.data_path, 'hadn_gesture_pose.csv'))
    df_train = df_train.merge(df_info[['pose_id', 'gesture_type', 'hand_type']], \
                                how='left', left_on='answer', right_on='pose_id')

    # 폴더별(Group)로 각 번호 부여
    df_train['groups'] = df_train['train_path'].apply(lambda x:x.split('/')[3])
    df_train.loc[:,:] = natsorted(df_train.values)
    # 노이즈 이미지 제거: 596번은 주먹쥐기 이미지인데 갑자기 손바닥을 펴는 노이즈 이미지가 5장있음 + 0번 폴더에 9번 이미지 역시 잘못된 클래스
    drop_idx = df_train[df_train['groups'].isin(['596'])].index.tolist()[3:8] + [9]  
    df_train = df_train.drop(drop_idx).reset_index(drop=True)  
    le = LabelEncoder()
    df_train['answer'] = le.fit_transform(df_train['answer'])
    
    # Split Fold
    # kf = StratifiedGroupKFold(n_splits=args.Kfold)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=args.seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y=df_train['answer'])):
        df_train.loc[val_idx, 'fold'] = fold
    df_val = df_train[df_train['fold'] == args.fold].reset_index(drop=True)
    df_train = df_train[df_train['fold'] != args.fold].reset_index(drop=True)
    
    # Augmentation
    self.train_transform = get_train_augmentation(img_size=args.img_size, ver=args.aug_ver)
    self.test_transform = get_train_augmentation(img_size=args.img_size, ver=1)
    
    ######################################################################
    # Flip Augmentation을 위한 Mapping dataframe
    df_info = pd.read_csv('../data/hand_gesture_pose.csv')
    df_info = df_info[df_info['hand_type'] != 'both']
    # drop idx, 동일한 약속, gesture_type, hand_type인데 다른 클래스인 경우 존재 -> 약속 1과 2로 이름을 나누어줌.
    df_info.loc[[105, 128], 'pose_name'] = '약속 1'  # idx: (105, 128)
    df_info.loc[[101, 124], 'pose_name'] = '약속 2'  # idx: (101, 124)

    # drop 41 idx, 동일한 약속, my hand, right class가 49와 54로 두 개있어 Mapping df만들 때 문제가 발생하여 미리 49번 클래스 처리
    df_info = df_info.drop(41)

    # Make a mapping dataframe
    df_info = df_info.groupby(['pose_name', 'view_type', 'gesture_type', 'hand_type']).sum().unstack().reset_index().dropna(axis=0)
    df_info['left'] = df_info.pose_id.left.apply(int)
    df_info['right'] = df_info.pose_id.right.apply(int)
    df_flip_info = df_info.drop('pose_id', axis=1).droplevel('hand_type', axis=1).reset_index(drop=True)
    print('Mapping dataframe Length', df_flip_info.shape)
    ######################################################################
    
    # TrainLoader
    self.train_loader = get_loader(df_train, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers, transform=self.train_transform, 
                                    df_flip_info=df_flip_info, flipaug_ratio=args.flipaug_ratio, label_encoder=le, margin=args.margin, random_margin=args.random_margin)
    self.val_loader = get_loader(df_val, batch_size=args.batch_size, shuffle=False,
                                    num_workers=args.num_workers, transform=self.test_transform)

    # Network
    self.model = Pose_Network(args).to(self.device)
    macs, params = get_model_complexity_info(self.model, (3, args.img_size, args.img_size), as_strings=True,
                                              print_per_layer_stat=False, verbose=False)
    self.logger.info('{:<30}  {:<8}'.format('Computational complexity: ', macs))
    self.logger.info('{:<30}  {:<8}'.format('Number of parameters: ', params))

    # Loss
    self.criterion = nn.CrossEntropyLoss()
    
    # Optimizer & Scheduler
    self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
    
    iter_per_epoch = len(self.train_loader)
    self.warmup_scheduler = WarmUpLR(self.optimizer, iter_per_epoch * args.warm_epoch)

    if args.scheduler == 'cos':
        tmax = args.tmax # half-cycle 
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max = tmax, eta_min=args.min_lr, verbose=True)
    elif args.scheduler == 'cycle':
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)

    
    if args.multi_gpu:
        self.model = nn.DataParallel(self.model).to(self.device)

    # Train / Validate
    best_loss = np.inf
    best_acc = 0
    best_epoch = 0
    early_stopping = 0
    start = time.time()
    for epoch in range(1, args.epochs+1):
      self.epoch = epoch

      if args.scheduler == 'cos':
        if epoch > args.warm_epoch:
          self.scheduler.step()

      # Training
      train_loss, train_acc = self.training(args)

      # Model weight in Multi_GPU or Single GPU
      state_dict= self.model.module.state_dict() if args.multi_gpu else self.model.state_dict()

      # Validation
      val_loss, val_acc = self.validate()

      # Save models
      if val_loss < best_loss:
          early_stopping = 0
          best_epoch = epoch
          best_loss = val_loss
          best_acc = val_acc

          torch.save({'epoch':epoch,
                      'state_dict':state_dict,
                      'optimizer': self.optimizer.state_dict(),
                      'scheduler': self.scheduler.state_dict(),
              }, os.path.join(save_path, 'best_model.pth'))
          self.logger.info(f'-----------------SAVE:{best_epoch}epoch----------------')
      else:
          early_stopping += 1

      # Early Stopping
      if early_stopping == args.patience:
          break

    self.logger.info(f'\nBest Val Epoch:{best_epoch} | Val Loss:{best_loss:.4f} | Val Acc:{best_acc:.4f}')
    end = time.time()
    self.logger.info(f'Total Process time:{(end - start) / 60:.3f}Minute')


  # Training
  def training(self, args):
    self.model.train()
    train_loss = AvgMeter()
    train_acc = 0

    scaler = grad_scaler.GradScaler()
    for i, (images, targets) in enumerate(tqdm(self.train_loader)):
      images = torch.tensor(images, device=self.device, dtype=torch.float32)
      targets = torch.tensor(targets, device=self.device, dtype=torch.long)
      
      if self.epoch <= args.warm_epoch:
          self.warmup_scheduler.step()

      self.model.zero_grad(set_to_none=True)
      if args.amp:
          with autocast():
              preds = self.model(images)
              loss = self.criterion(preds, targets)
          scaler.scale(loss).backward()

          # Gradient Clipping
          if args.clipping is not None:
              scaler.unscale_(self.optimizer)
              nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)

          scaler.step(self.optimizer)
          scaler.update()

      else:
        preds = self.model(images)
        loss = self.criterion(preds, targets)
        loss.backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), args.clipping)
        self.optimizer.step()

      if args.scheduler == 'cycle':
        if self.epoch > args.warm_epoch:
          self.scheduler.step()

      # Metric
      train_acc += (preds.argmax(dim=1) == targets).sum().item()
      # log
      train_loss.update(loss.item(), n=images.size(0))
        
    train_acc /= len(self.train_loader.dataset)

    self.logger.info(f'Epoch:[{self.epoch:03d}/{args.epochs:03d}]')
    self.logger.info(f'Train Loss:{train_loss.avg:.3f} | Acc:{train_acc:.4f}')
    return train_loss.avg, train_acc
        
  # Validation or Dev
  def validate(self):
    self.model.eval()
    with torch.no_grad():
      val_loss = AvgMeter()
      val_acc = 0

      for _, (images, targets) in enumerate(self.val_loader):
        images = torch.tensor(images, device=self.device, dtype=torch.float32)
        targets = torch.tensor(targets, device=self.device, dtype=torch.long)

        preds = self.model(images)
        loss = self.criterion(preds, targets)

        # Metric
        val_acc += (preds.argmax(dim=1) == targets).sum().item()
        # log
        val_loss.update(loss.item(), n=images.size(0))
      val_acc /= len(self.val_loader.dataset)
      self.logger.info(f'Valid Loss:{val_loss.avg:.3f} | Acc:{val_acc:.4f}')
    return val_loss.avg, val_acc

### Stratified GroupKFold Split
* 데이터셋을 보면 총 이미지는 5888개 이지만, 한 폴더 내의 이미지가 대부분 비슷한 것을 확인할 수 있다. 

* 이러한 데이터를 일반적인 split방법으로 나눌 경우 validation accuracy가 거의 1에 가깝게 나오게 되어, 어떤 class에 대한 적중률이 떨어지는 지에 대한 분석을 진행할 수 없다. 

* 따라서, 각 폴더별로 Group을 부여하고 해당 Group 전체가 Train or Valid로 들어가게 하는 GroupKFold를 사용하고, 추가적으로 Class Label 분포를 고려하기 위해 Stratified KFold를 함께 적용한다.

* 단일 모델로 public 기준 0.024의 스코어를 얻음

* 그 이후 Fold를 다르게 학습한 경우 Public score가 0.15~0.20 정도로 매우 극심한 차이를 보임

* 확인해보니 전체 데이터 중 특정 클래스가 1,2개 폴더만 존재하는 클래스가 있었음. groupkfold로 split시에 특정 클래스는 train에만, 또는 Valid에만 들어가는 것을 확인. 이는 특정 클래스에 대해 검증을 수행할 수 없게 되는 문제가 있음. 

* 이를 방지하기 위해 make dataset section에서 볼 수 있듯, 폴더가 1개인 클래스를 폴더가 2개 이상이 되도록 만들어 줌. 이 때 단순히 augmentation을 사용한 것이 아니라 flip augmentation(+rotation)을 수행하였음. 

### Validation prediction의 분포 확인
* 위 방법을 통해 적절한 train/ valid split을 수행한 후에도 여전히 fold별 편차가 심한 것을 확인. 
* -> 모델이 어떤 클래스를 못 맞추었는지에 대한 분석 진행
* 분석 결과 특정 몇 개 클래스에서 logloss 값이 매우 크게 발생하는 것을 확인하였고, insight를 얻기 위해 해당 클래스들의 이미지를 살펴봄.

In [1]:
## Case1
# img = Image.open("./etc/숫자1_검지흔들기.png")
# plt.figure(figsize=(10,5))
# plt.imshow(img)

In [ ]:
## Case2
# img = Image.open("./etc/주먹내밀기_주먹쥐기.png")
# plt.figrue(figsize=(10,5))
# plt.imshow(img)

In [ ]:
# 시각화를 위한 function
def visualize(folder_num):
  path = f'../data/train/{folder_num}/*.png'
  image_list = glob(path)
  length = len(image_list)

  fig, ax = plt.subplots(1,length, figsize=(50,10))
  for i, image in enumerate(image_list):
    image = Image.open(image).convert('RGB')
    ax[i].imshow(image)
  plt.show()

In [ ]:
df =pd.read_csv(opj(args.data_path, "df_train.csv"))
df['groups'] = df['train_path'].apply(lamda x:x.split("/")[3])
df = df.drop_duplicates("groups")

(Case 1) 첫 번째는 숫자1과 부정(검지 흔들기) 대해 큰 Logloss값이 발생.
두 이미지는 각각 숫자1과 부정(검지 흔들기) Class임.




In [ ]:
path = f'../data/train/'
number1_folder = df[df['answer_name'] == '숫자1']['groups'].tolist()
shake_folder = df[df['answer_name'] == '부정(검지 흔들기)']['groups'].tolist()

image1 = Image.open(opj(path, number1_folder[0], '1.png'))   # 352번 폴더
image2 = Image.open(opj(path, shake_folder[11], '1.png'))    # 489번 폴더
print(number1_folder[0], shake_folder[11])

fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(image1)
ax[1].imshow(image2)
plt.show()

당 두 개의 클래스는 사람이 봐도 구분이 힘듦.
마찬가지로 사람도 구분 할 수 없는 이미지를 모델이 각각 다른 Class로 학습을 하게되어, 모델이 해당 클래스 샘플들에 대해 매우 헷갈려하여 Logloss 값이 크게 나오는 것을 확인.


(Case 2) 두 번째는 주먹쥐기와 경고(주먹 내밀기) 대해서도 큰 Logloss값이 발생. 

주먹쥐기와 경고(주먹 내밀기) Class.


In [ ]:
path = f'../data/train/'
number1_folder = df[df['answer_name'] == '주먹쥐기']['groups'].tolist()
shake_folder = df[df['answer_name'] == '경고(주먹 내밀기)']['groups'].tolist()

image1 = Image.open(opj(path, number1_folder[1], '1.png'))   
image2 = Image.open(opj(path, shake_folder[9], '1.png'))    
print(number1_folder[1], shake_folder[9])

fig, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(image1)
ax[1].imshow(image2)
plt.show()

* (숫자1, 검지흔들기)pair와 마찬가지로 단순히 두 이미지만 봤을때는 어떤 클래스인지 판단하기가 어려움.
* 주먹쥐기의 경우에는 계속 같은 위치에서 크게 움직임이 없지만 주먹 내밀기의 경우에는 주먹이 카메라에 가까워졌다 멀어졌다 하는식으로 촬영된 것을 확인.
* 위의 2가지 Case를 해결하기 위한 2가지 방법.
  1.  폴더 내 이미지들의 Sequence를 살려서, 한 번의 입력에 여러 장의 이미지가 들어가 모델이 학습하도록 하는 방법
    *   위 방법은 (a)시퀀스를 어느 정도의 길이로 만들어주느냐, (b)각 폴더 내에서 시퀀스 Pair를 얼마나 잘 만들어주느냐(변화가 있는 Class의 경우) 같은 Pre-processing에 영향을 크게 받을 것으로 추측.
  2. Keypoint를 사용한 Rule에 기반한 접근 방법
    * 본 대회는 Train과 Test모두 입력으로 이미지뿐만 아니라 Keypoint를 사용할 수 있음. 따라서, 기존에 학습했던 것처럼 모델의 학습과 추론을 수행함.
    * 다만, 추론 시에 (숫자1, 검지 흔들기), (주먹쥐기, 주먹 내밀기) 두 Case 중에 하나의 Class를 예측하면 Keypoint를 사용한 Rule로 처리하는 방법임.
    * 예를 들어, 모델이 숫자1(왼손, My View) 또는 검지 흔들기(왼손, My View)로 예측하였으면, Keypoint에 기반한 알고리즘으로 넘어가게되고 해당 알고리즘을 거친 예측 정답이 나오게 됨.
    * 추가적으로, 해당 Task는 같은 숫자1이라도 왼손인지 오른손인지, 또는 My View인지 Your View에 따라 Class가 다름.
    * 모델이 왼손인지 오른손인지, 또는 My View인지 Your View인지는 맞추었을 것이라 판단하였고, (숫자1(왼손, My View) - 검지 흔들기(왼손, My View))과 같이 좌우, View타입은 동일하게 매핑시킴.



### Post-Processing in detail (Method)


Case1
숫자1과 부정(검지 흔들기)의 경우, 한 폴더 내에서 숫자1은 검지의 움직임이 크지 않지만 검지 흔들기는 검지를 흔들어야하기 때문에 검지 Keypoint의 X좌표로의 움직임이 상대적으로 클 수 밖에 없음.
따라서, 손가락(검지)의 가장 위에 있는(0,0픽셀을 기준으로 Y값이 가장 작은) keypoint의 움직임을 통해 구별하고자 함.
예를들어, 폴더내 이미지가 10개가 있다면 10개의 검지 Keypoint 좌표가 있게 되고 해당 x좌표들에 대해서 max-min을 계산하게 된다. 즉, 각 폴더내에서 1개의 x좌표 변화량이 계산 됨.
두 클래스에 대해 계산된 검지 x좌표 변화량들에 대해서 적절한 threshold값을 설정해 클래스를 구분짓게 함.

Case2
주먹 쥐기와 주먹 내밀기 같은 경우 손의 움직임을 통해 구별하고자 함.
이는 단순하게 keypoint중 가장 오른쪽 x좌표의 변화량으로 계산함.
Case1과 마찬가지로 keypoint의 x좌표 max-min을 계산.
두 클래스에 대해 계산된 x좌표 변화량들에 대해서 적절한 threshold값을 설정해 두 클래스를 구분짓게 함.

In [ ]:
# threshold를 계산하기 위해 각 case에 대한 변화량을 계산하는 함수 정의
def check_stats(find_list, ver):
  train_path = '../data/train'
  train_folders = natsorted(glob(train_path + '/*'))
  stat_list = []
  for _, train_folder in tqdm(enumerate(train_folders)):
    try:
      json_path = glob(train_folder + '/*.json')[0]
      js = json.load(open(json_path))
      cat = js.get('action')[0]
      keypoints = js['annotations']
      keypoints = np.array([point['data'] for point in keypoints])  # (N-이미지개수, 21 or 42(keypoints), 3(x,y,z 좌표))
    except:
      pass
    if cat in find_list:
      # 숫자1과 검지흔들기 구분 # case1
      # 검지는 이미지내 keypoints들 중 가장 작은 y값(이미지 상 가장 높은 위치)을 갖는 point임. 
      # 해당 point의 x값을 뽑음.
      if ver ==1 : 
        keypoints = keypoints[:, :, :2]  # keypoints : (N, 21 or 42, 2)
        x_I_finger = [point_per_img[:,0][point_per_img[:,1].argmin()] for point_per_img in keypoints] # (N, 2) Y축으로 가장 작은 포인트 두개 추출
        stat_list.append(np.max(x_I_finger) - np.min(x_I_finger))
      
      # 주먹쥐기와 주먹 내밀기(경고) # Case2
      # keypoints들 중 가장 큰 x값(이미지 상 가장 우측 위치)을 갖는 point임.
      # Case2같은 경우는 left 손목이 없기때문에 해당 logic이 잘 작동함.
      elif ver == 2:
        keypoints = keypoints[:, :, 0]   
        x_values = [point_per_img[point_per_img.argmax()] for point_per_img in keypoints]  # 가장 오른쪽
        stat_list.append(np.max(x_values) - np.min(x_values))

  print(stat_list)
  return stat_list

##label##
######### ver1 ############
find_list0 = [0, 10, 100, 110] # ['숫자 1', '숫자1']  my hand, your hand 좌우
find_list1= [42, 67, 142, 167] # ['부정(검지 흔들기)'] my hand, your hand 좌우

##########ver2 ############
find_list2 = [146] # ['주먹쥐기']  Your hand 우
find_list3 = [163] # ['경고(주먹 내밀기)'] Your hand 우

find_list4 = [171] # ['주먹쥐기']  Your hand Both
find_list5 = [191] # ['경고(주먹 내밀기)'] Your hand Both

Case1
* Train dataset 내 두 클래스간 검지 위치 변화량 차이가 분명함을 확인.
* 부정(검지 흔들기)클래스의 특정 폴더에서 9.58, 20.37과 같은 작은 값들도 존재하였는데, 해당 폴더를 살펴보면 손가락의 움직임이 거의 없어 숫자 1이라고 판단해도 될만한 노이즈인 경우가 있었음.
* 따라서, 해당 threshold로는 완벽한 구분이 불가능 할 수 있지만, 두 Outlier를 제외하면 움직임 값이 매우 크기 때문에 숫자1의 움직임들 중 Max값인 26.85에  Margin(5)을 적용시킴.

In [ ]:
# 숫자1 & 검지 흔들기
li0 = check_stats(find_list0,1) #숫자1 or 숫자 1
li1 = check_stats(find_list1,1) #부정(검지 흔들기)
threshold_ver1 = max(li0) + 5   # Margin 5
print(f'\n{threshold_ver1:.3f}보다 크면 부정(검지 흔들기) 클래스')

Case2
* 두 클래스간 가장 큰 x좌표의 위치 변화량 차이가 분명함을 확인할 수 있음.
* 주먹쥐기(596번 폴더) 클래스이지만 주먹을 활짝 펴버리는(첫번째 이미지) 노이즈 이미지가 존재함. 해당 이미지 때문에 596번 폴더의 x좌표 변화량이 매우 크게 나타남을 확인.
* 596번 폴더는 제외하고 threshold를 구함.



In [ ]:
# 주먹쥐기 vs 주먹 내밀기 Right
li2 = check_stats(find_list2,2)   
li3 = check_stats(find_list3,2)   
threshold_ver2 = max(li2) + 5   # Margin 5
print(f'\n{threshold_ver2:.3f}보다 크면 주먹 내밀기(right) 클래스')

# 주먹쥐기 vs 주먹 내밀기 Both
li4 = check_stats(find_list4,2)   
li4 = li4[1:]  # 596번 폴더 변화량(218.313) Outlier -> 제외
li5 = check_stats(find_list5,2)   
threshold_ver2_both = max(li4) + 5  # Margin 5
print(f'\n{threshold_ver2_both:.3f}보다 크면 주먹 내밀기(both) 클래스')

## Case2에서 Both와 Right를 구분하여 threshold를 구하고자 했지만 큰 차이가 없어 실제 inference시에는 통합함. ###

### Stratified KFold
* 앞서 Stratified GroupKFold로 데이터를 Split하여 오류가 심한 클래스에 대한 분석을 수행함.
* 다만, Stratified GroupKFold 방법은 여전히 각 클래스당 폴더 개수가 너무 적기 때문인지 Validation Loss 기준으로 더 좋은 성능을 얻지는 못하였음.
* 그래서 처음에 StratifiedGroupKFold를 사용하고자 했던 선택을 바꾸어, 클래스 비율을 고려하여 랜덤하게 섞어주는 Stratified KFold를 사용.

### Main Function

In [ ]:
def main(args):
  # Random Seed
  seed = args.seed
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = True

  save_path = os.path.join(args.model_path, (args.exp_num).zfill(3))
  # Create model directory
  os.makedirs(save_path, exist_ok=True)
  Trainer(args, save_path)

In [ ]:
if __name__ == '__main__':
  for i in range(5): # 5Folds Training
    args.fold = i
    args.exp_num = str(i)
    main(args)

### Inference with Ensemble
* 위에서 구한 threshold를 이용하여 Rule Base inference를 구축.
* 변수 replace_dict를 통해 헷갈리는 두 클래스를 매칭함.


In [ ]:
# test의 keypoints(json) 변화량을 구하기 위한 함수 정의
def Refiner(keypoints, ver):
  keypoints = np.array([point['data']for point in keypoints])
  # 숫자 1과 검지 흔들기 구분
  if ver == 1:  
    keypoints = keypoints[:, :, :2]  
    x_I_finger = [point_per_img[:,0][point_per_img[:,1].argmin()] for point_per_img in keypoints]
    query_value = np.max(x_I_finger) - np.min(x_I_finger)
    
  # 주먹쥐기와 주먹 내밀기(경고) 
  elif ver == 2:
    keypoints = keypoints[:, :, 0]  
    x_values = [point_per_img[point_per_img.argmax()] for point_per_img in keypoints]
    query_value = np.max(x_values) - np.min(x_values)

  return query_value

In [ ]:
# Download the Pretrained Weight (./results/ 경로)
os.makedirs('./results/', exist_ok=True)
!wget -i https://raw.githubusercontent.com/wooseok-shin/Egovision-1st-place-solution/main/load_pretrained.txt -P results   

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_path = f'../data/test' 
test_folders = natsorted(glob(test_path + '/*'))

args = easydict.EasyDict({'encoder_name':'regnety_040',
                        'drop_path_rate':0,
                        })

load_pretrain = True  # Use Pretrained weights
ensemble_test = True  # Ensemble or Single
refine = True         # Use Refiner (Rule-base)


if load_pretrain:  # Github로부터 Pretrained Weight Load
  model_path0 = './results/0Fold_model.pth' # fold0
  model_path1 = './results/1Fold_model.pth' # fold1
  model_path2 = './results/2Fold_model.pth' # fold2
  model_path3 = './results/3Fold_model.pth' # fold3
  model_path4 = './results/4Fold_model.pth' # fold4

else:  # 위에서 학습한 모델 Weight Load
  model_path0 = './results/000/best_model.pth' # fold0
  model_path1 = './results/001/best_model.pth' # fold1
  model_path2 = './results/002/best_model.pth' # fold2
  model_path3 = './results/003/best_model.pth' # fold3
  model_path4 = './results/004/best_model.pth' # fold4


# 5Fold Ensemble
if ensemble_test:
  model0 = Pose_Network(args).to(device)
  model0.load_state_dict(torch.load(model_path0)['state_dict'])
  model0.eval()

  model1 = Pose_Network(args).to(device)
  model1.load_state_dict(torch.load(model_path1)['state_dict'])
  model1.eval()

  model2 = Pose_Network(args).to(device)
  model2.load_state_dict(torch.load(model_path2)['state_dict'])
  model2.eval()

  model3 = Pose_Network(args).to(device)
  model3.load_state_dict(torch.load(model_path3)['state_dict'])
  model3.eval()

  model4 = Pose_Network(args).to(device)
  model4.load_state_dict(torch.load(model_path4)['state_dict'])
  model4.eval()

  model_list = [model0, model1, model2, model3, model4]

else:  # Single Best Model (Using the pretrained weight)
  model_path = './results/single_best_model.pth'
  single_best = Pose_Network(args).to(device)
  single_best.load_state_dict(torch.load(model_path)['state_dict'])
  single_best.eval()
  model_list = [single_best]


img_size = 288
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((img_size, img_size)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        ])

sub = pd.read_csv('../data/sample_submission.csv')
df_info = pd.read_csv('../data/hand_gesture_pose.csv')
le = LabelEncoder()
le.fit(df_info['pose_id'])
trans = le.transform

# Class Mapping dict
ver1_list = trans([0, 42, 10, 67, 100, 142, 110, 167])   
ver2_list = trans([146, 163, 171, 191])
replace_dict = {146:163, 171:191, 0:42, 10:67, 100:142, 110:167}
replace_dict = dict([trans(x) for x in list(replace_dict.items())])   # Mapping (Origin:0~195 to 0~156)

total_list = np.concatenate([ver1_list, ver2_list]).tolist()


for i, test_folder in tqdm(enumerate(test_folders)):
  dir = os.path.dirname(test_folder)
  folder_num = os.path.basename(test_folder)
  json_path = opj(dir, folder_num, folder_num+'.json')
  js = json.load(open(json_path))
  keypoints = js['annotations']  # 해당 이미지에 해당하는 Keypoints
  images_list = natsorted(glob(test_folder + '/*.png'))
  images = []
  for _, (point, image_name) in enumerate(zip(keypoints, images_list)):
    croped_image = crop_image(image_name, point, margin=100)
    image = transform(croped_image)
    images.append(image)

  images = torch.stack(images).to(device)
  ensemble = np.zeros((157,), dtype=np.float32)
  for model in model_list:
    preds = model(images)
    preds = torch.softmax(preds, dim=1)
    preds = torch.mean(preds, dim=0).detach().cpu().numpy()    # shape:(157,)
    ensemble += preds
  preds = ensemble / len(model_list)
  pred_class = preds.argmax().item()
  if refine and (pred_class in total_list):
    idx = list(replace_dict.keys()).index(pred_class) if pred_class in replace_dict.keys() else list(replace_dict.values()).index(pred_class)
    cand1, cand2 = list(replace_dict.items())[idx]

      if pred_class in ver1_list:
        query_value = Refiner(keypoints, ver=1)
        answer = cand1 if query_value < threshold_ver1 else cand2

      elif pred_class in ver2_list:
        query_value = Refiner(keypoints, ver=2)
        answer = cand1 if query_value < threshold_ver2_both else cand2

      preds[answer] = 1
      preds = np.where(preds != 1, 0, preds)  # Refiner를 통해 나온 class를 제외한 나머지의 확률값은 모두 0으로 변환

  sub.iloc[i, 1:] = preds.astype(float)

sub.to_csv('./results/submission_train_add_ensemble_rule.csv',index=False)